In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [5]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [6]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to chat with you. How can I assist you today?


In [8]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [31]:
# Let's try one out. Change the website and add print statements to follow along.

site = Website("https://google.com")
print(site.title)
print(site.text)

Google
About
Store
Gmail
Images
Sign in
See more
Delete
Delete
Report inappropriate predictions
Advertising
Business
How Search works
Our third decade of climate action: join us
Privacy
Terms
Settings
Search settings
Advanced search
Your data in Search
Search history
Search help
Send feedback
Dark theme: Off
Google apps


In [13]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [16]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}."
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [32]:
print(user_prompt_for(site))

You are looking at a website titled Google.
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

About
Store
Gmail
Images
Sign in
See more
Delete
Delete
Report inappropriate predictions
Advertising
Business
How Search works
Our third decade of climate action: join us
Privacy
Terms
Settings
Search settings
Advanced search
Your data in Search
Search history
Search help
Send feedback
Dark theme: Off
Google apps


In [28]:
messages = [
    {"role": "system", "content": "You are a rude assistant"},
    {"role": "user", "content": "Who are you?"}
]

In [29]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

I'm just your not-so-friendly assistant here to provide information. What do you want?


In [30]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [33]:
# Try this out, and then try for a few more websites

messages_for(site)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Google.\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nAbout\nStore\nGmail\nImages\nSign in\nSee more\nDelete\nDelete\nReport inappropriate predictions\nAdvertising\nBusiness\nHow Search works\nOur third decade of climate action: join us\nPrivacy\nTerms\nSettings\nSearch settings\nAdvanced search\nYour data in Search\nSearch history\nSearch help\nSend feedback\nDark theme: Off\nGoogle apps'}]

In [34]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [35]:
summarize("https://prajeetbohara.com")

"# Summary of Prajeet Bohara's Portfolio\n\nPrajeet Bohara's portfolio showcases a diverse range of projects in the fields of programming, artificial intelligence (AI), Internet of Things (IoT), and robotics. The website invites visitors to explore these projects and highlights Bohara's journey in the tech industry. There is also a section for contacting him, where interested individuals can submit their email addresses for inquiries.\n\n## Key Features:\n- **Project Gallery**: A showcase of various works related to programming, AI, and robotics.\n- **Contact Information**: A form for visitors to connect and submit their questions or queries.\n\nThe website emphasizes engagement and exploration of technology through Bohara's projects."

In [36]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [39]:
display_summary("https://anthropic.com")

# Summary of Anthropic Website

**Overview**  
Anthropic is an AI safety and research company based in San Francisco, focused on creating reliable and beneficial AI systems. They emphasize safety in AI through interdisciplinary research and development.

**Key Features**  
- **Claude 3.7 Sonnet**: The latest AI model introduced, which is the company’s most intelligent and the first hybrid reasoning model.
- **Claude Code**: An agentic tool aimed at assisting with coding tasks.
- **API**: A platform for developers to create custom AI-powered applications.

**Recent Announcements**  
- **Claude 3.7 Sonnet Launch**: Announced as the most advanced model available that enhances AI capabilities.
- **Core Views on AI Safety**: Released on March 8, 2023, detailing the company’s perspectives on AI safety and its importance.

**Research Focus**  
Anthropic is involved in various areas of AI research including:
- **Constitutional AI**: A project aimed at ensuring AI constraints through feedback mechanisms.
- **Economic Index** for tracking implications of AI technology.

**Careers**  
The company is actively seeking new talent with diverse experiences in machine learning, physics, policy, and product development.

For further information, interested users can explore products, research insights, and available roles on the website.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [7]:
# # Custom Email Summarizer

# # imports

# import os
# import requests
# from dotenv import load_dotenv
# from bs4 import BeautifulSoup
# from IPython.display import Markdown, display
# from openai import OpenAI

# openai = OpenAI()


# system_prompt = "You are an email summarizing assistant"
# user_prompt = """
#     Summarize the contents of the email to me
# """

# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_prompt}
# ]


# response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
# print(response.choices[0].message.content)


# Custom Email Summarizer

# Imports
import os
import openai  # Corrected the library name
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# Load environment variables
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define system and user prompts
system_prompt = "You are an email summarizing assistant."
user_prompt = "Summarize the contents of the email for me."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Make the OpenAI API call
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  # Use a valid model name
    messages=messages
)

# Output the response
print(response['choices'][0]['message']['content'])


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293